<a href="https://colab.research.google.com/github/dapopadev/project/blob/main/Algoritmo%20k-means%20and%20BsKMeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=bfacee3b42dc6185f9c6f20ab232dbf33d561bf1a9d8120fd985e83d32295147
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark






**2.INSTALAMOS EL MÓDULO PYSPARK**



3.**CREAMOS LA SESIÓN PYSPARK***

In [2]:
from pyspark.sql import SparkSession

In [3]:
from google.colab import drive

 4. **DISEÑAMOS UNA UNIDAD PERSONAL DE GOOGLE DRIVE**

In [4]:
drive.mount("/drive")

Mounted at /drive


In [5]:
spark=SparkSession.builder.appName("my_app").getOrCreate()

5.***CARGAMOS EL ARCHIVO STOCKS_2021***

In [6]:
df=spark.read.option("Header",True).csv("/drive/MyDrive/CSV_stocks_2021.csv")

 6.**IMPORTAREMOS LOS OBJETOS K-MEANS Y VECTORASSEMBLER DEL MÓDULO PYSPARK.ML**

In [7]:
from pyspark.ml.clustering import KMeans

In [8]:
from pyspark.ml.feature import VectorAssembler

**7.MODIFICAMOS EL TIPO DE DATOS DE LAS COLUMNAS OPEN,LOW Y CLOSE A FLOAT**

In [9]:
df=df.withColumn('open',df.open.cast('float'))

In [10]:
df=df.withColumn('low',df.low.cast('float'))

In [11]:
df=df.withColumn('close',df.close.cast('float'))

In [12]:
df.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- open: float (nullable = true)
 |-- high: string (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- volume: string (nullable = true)
 |-- dividends: string (nullable = true)
 |-- stock splits: string (nullable = true)
 |-- date: string (nullable = true)
 |-- ccy: string (nullable = true)



**8.CREAREMOS UNA COLUMNA'FEATURES' UTILIZANDO LAS COLUMNAS OPEN,LOW,Y CLOSE MEDIANTE VECTORASSEMBLER**

In [13]:
va=VectorAssembler(inputCols=['open','low','close'],outputCol='features')

In [14]:
va_df=va.transform(df)

In [15]:
va_df.show()

+------+---------+------------------+---------+---------+-------+---------+------------+----------+---+--------------------+
|ticker|     open|              high|      low|    close| volume|dividends|stock splits|      date|ccy|            features|
+------+---------+------------------+---------+---------+-------+---------+------------+----------+---+--------------------+
|   HON|207.45502|209.42141232111007|206.14735|209.12645|1406400|      0.0|           0|2020-12-31|USD|[207.455017089843...|
|   HON| 209.2641|209.43123817541044|202.89297|204.45625|2328900|      0.0|           0|2021-01-04|USD|[209.264099121093...|
|   HON|203.50256| 206.6586329757187|203.50256| 204.9577|2172100|      0.0|           0|2021-01-05|USD|[203.502563476562...|
|   HON|205.93106|210.38495140642195|205.71475|208.69385|2747900|      0.0|           0|2021-01-06|USD|[205.931060791015...|
|   HON|209.31328|210.41445198270796|207.25839|209.03798|2057300|      0.0|           0|2021-01-07|USD|[209.313278198242...|


**9.CREAREMOS EL OBJETO K-MEANS Y LO VAMOS A CONFIGURAR PARA ESTABLECER 5 CLUSTERS'**

In [16]:
kmeans=KMeans(k=5)

In [17]:
model=kmeans.fit(va_df.select('features'))

In [18]:
transformed=model.transform(va_df)

In [19]:
import matplotlib.pyplot as plt


**10.ENTRENAMOS EL MODELO Y VAMOS A MOSTRAR LOS RESULTADOS DE NUESTRA PREDICCIÓN**

In [20]:
from pyspark.ml.clustering import BisectingKMeans

In [21]:
bkm=BisectingKMeans().setK(5).setSeed(1)

In [22]:
model=bkm.fit(va_df)

In [23]:
predictions=model.transform(va_df)

In [24]:
predictions.show(10)

+------+---------+------------------+---------+---------+-------+---------+------------+----------+---+--------------------+----------+
|ticker|     open|              high|      low|    close| volume|dividends|stock splits|      date|ccy|            features|prediction|
+------+---------+------------------+---------+---------+-------+---------+------------+----------+---+--------------------+----------+
|   HON|207.45502|209.42141232111007|206.14735|209.12645|1406400|      0.0|           0|2020-12-31|USD|[207.455017089843...|         1|
|   HON| 209.2641|209.43123817541044|202.89297|204.45625|2328900|      0.0|           0|2021-01-04|USD|[209.264099121093...|         1|
|   HON|203.50256| 206.6586329757187|203.50256| 204.9577|2172100|      0.0|           0|2021-01-05|USD|[203.502563476562...|         1|
|   HON|205.93106|210.38495140642195|205.71475|208.69385|2747900|      0.0|           0|2021-01-06|USD|[205.931060791015...|         1|
|   HON|209.31328|210.41445198270796|207.25839|2